In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd

train_df = pd.read_csv('/content/drive/MyDrive/dataset/cls_train_data.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/dataset/cls_validation_data.csv')

train_df.head()

,text,empathy
0,"엄마, 아기가 태어나니까 내가 부모로서 해야 할 게 참 많은 것 같아요.",3
1,어제 평소보다도 격하게 막 온몸을 써가면서 울더라고요. 얼마나 당황했는지 몰라요.,5
2,맞아요. 젖을 물려주니 금세 뚝 그쳤어요. 난 분명히 순했을 것 같은데.,5
3,잠도 못 자고 우는 아이 달래랴 수유하랴 머리카락이 다 빠지는 줄 알았어요.,3
4,그러면서 엄마 생각이 많이 났어요. 엄마는 직장까지 다니면서 나를 키우느라 얼마나 ...,2


In [5]:
from transformers import BertTokenizerFast

Q_TKN = "<Q>"
A_TKN = "<A>"
BOS = '</s>'
EOS = '</s>'
UNK = '<unk>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'

# KoGPT2 토크나이저 로드
tokenizer = BertTokenizerFast.from_pretrained('monologg/distilkobert')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizerFast'.


In [6]:
from datasets import Dataset
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

label_classes = ['1', '2', '3', '4', '5']
mlb = MultiLabelBinarizer(classes=label_classes)

def get_cls_input(examples):
    inputs = examples['text']
    labels = [label.split(',') for label in examples['empathy']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    labels_binary = mlb.fit_transform(labels)
    labels = labels_binary.astype(np.float32).tolist()

    model_inputs["labels"] = labels
    return model_inputs

# 데이터셋 전처리
train_inputs = train_dataset.map(get_cls_input, batched=True)
valid_inputs = valid_dataset.map(get_cls_input, batched=True)


Map:   0%|          | 0/181802 [00:00<?, ? examples/s]

Map:   0%|          | 0/22554 [00:00<?, ? examples/s]

In [8]:
train_inputs

Dataset({
    features: ['text', 'empathy', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 181802
})

In [9]:
valid_inputs

Dataset({
    features: ['text', 'empathy', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 22554
})

In [10]:
import wandb

wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [11]:
# method
sweep_config = {
    'method': 'random'
}

# hyperparameters
parameters_dict = {
    'lr_scheduler_type':{
        'values': ['linear', 'cosine', 'polynomial']
    },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
    },
    'weight_decay': {
        'values': [0.1, 0.3, 0.5]
    },
    'train_batch_size': {
        'values': [8, 16, 32]
    },
    'eval_batch_size': {
        'values': [8, 16, 32]
    }
}

sweep_config['parameters'] = parameters_dict

In [12]:
from peft import LoraConfig, TaskType, get_peft_model

lora_config = LoraConfig(
    task_type = "SEQ_CLS", #시퀀스 분류 작업을 위한 설정
    inference_mode = False, #모델이 추론모드인지 여부 설정
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['attention.q_lin', 'attention.k_lin', 'attention.v_lin', 'attention.out_lin', 'ffn.lin1', 'ffn.lin2']
)

In [13]:
import evaluate

def compute_metrics(pred):
    logits, labels = pred
    predictions = (logits > 0).astype(float)  # 시그모이드 활성화 함수 적용 후 임계값 0.5 기준으로 이진
    labels = labels.flatten()
    predictions = predictions.flatten()

    # 정확도, F1 score 계산
    acc_metrics = evaluate.load("accuracy")
    accuracy = acc_metrics.compute(predictions=predictions, references=labels)

    f1_metrics = evaluate.load("f1")
    f1 = f1_metrics.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy, "f1_metrics": f1}

In [14]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from accelerate import Accelerator

def train():
  wandb.init()
  config = wandb.config

  # DistilKoBert 모델 로드
  model = DistilBertForSequenceClassification.from_pretrained(
      'monologg/distilkobert',
      num_labels=5,
      problem_type="multi_label_classification",
  )
  accelerator = Accelerator() #데이터 병렬 처리 원활
  model = accelerator.prepare(model)
  # lora 적용
  model = get_peft_model(model, lora_config)

  # 학습 설정
  training_args = TrainingArguments(
      fp16=True,
      output_dir='./results',
      num_train_epochs=4,
      lr_scheduler_type=config.lr_scheduler_type,
      learning_rate=config.learning_rate,
      per_device_train_batch_size=config.train_batch_size,
      per_device_eval_batch_size=config.eval_batch_size,
      warmup_steps=10000,
      weight_decay=config.weight_decay,
      logging_dir='./logs',
      logging_steps=0.01,
      do_eval=True,
      eval_strategy="steps",
      eval_steps=0.1,
      remove_unused_columns=True,
  )

  # Trainer 설정
  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_inputs,
      eval_dataset=valid_inputs,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  model.save_pretrained('./distilKoBERT-classification')
  tokenizer.save_pretrained('./distilKoBERT-classification')

In [15]:
# 캐시 지우기
import torch, gc

gc.collect()
torch.cuda.empty_cache()

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity='nkim123', project='minidlthon_distilKoBERT_classification')

wandb.agent(sweep_id, train, count=1)

wandb.finish()

Create sweep with ID: fus9sqlp
Sweep URL: https://wandb.ai/nkim123/minidlthon_distilKoBERT_classification/sweeps/fus9sqlp


wandb: Agent Starting Run: oxqwdlep with config:
wandb: 	eval_batch_size: 32
wandb: 	learning_rate: 3.60063074924359e-05
wandb: 	lr_scheduler_type: linear
wandb: 	train_batch_size: 32
wandb: 	weight_decay: 0.3
wandb: Currently logged in as: nkim12 (nkim123). Use `wandb login --relogin` to force relogin


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'lr_scheduler_type' was locked by 'sweep' (ignored update).


Step,Training Loss,Validation Loss


In [ ]:
# 저장된 모델 및 토크나이저 로드
trained_tokenizer = BertTokenizerFast.from_pretrained('./distilKoBERT-classification')

lora_config = LoraConfig(
    task_type = "SEQ_CLS", #시퀀스 분류 작업을 위한 설정
    inference_mode = True, #모델이 추론모드인지 여부 설정
    r=8, #낮은 랭크값을 사용하여 파라미터 수를 줄인다.
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['attention.q_lin', 'attention.k_lin', 'attention.v_lin', 'attention.out_lin', 'ffn.lin1', 'ffn.lin2']
)
model = DistilBertForSequenceClassification.from_pretrained(
      './distilKoBERT-classification',
      num_labels=5,
      problem_type="multi_label_classification",
      )
accelerator = Accelerator() #데이터 병렬 처리 원활
model = accelerator.prepare(model)
# lora 적용
trained_model = get_peft_model(model, lora_config)

In [ ]:
import torch

def predict(text, model, tokenizer, num_classes=5, threshold=0.6):
    # 모델을 평가 모드로 전환
    model.eval()

    # 입력 문장 토큰화
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)

    # 모델에 입력을 전달하여 로짓(logits)을 얻음
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # 로짓에 시그모이드 적용하여 확률로 변환
    probabilities = torch.sigmoid(logits)
    # 임계값을 기준으로 이진화
    predictions = (probabilities > threshold).int()

    # 레이블 디코딩
    label_classes = ['조언', '격려', '위로', '동조', '']
    predicted_labels = [label_classes[i] for i in range(num_classes) if predictions[0][i] == 1]

    return predicted_labels


In [ ]:
# 예제 입력 문장
input_text = "오늘 뭐 해?"

# 분류 결과 추론
# threshold 잘 설정해야
predicted_labels = predict(input_text, trained_model, trained_tokenizer)
print(f"Predicted labels: {predicted_labels}")